In [ ]:
 # Veri yapıları ile çalışmak için pandas modülü.
from sklearn import metrics
import time
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import RandomOverSampler



In [ ]:
df = pd.read_excel('df_5_toplam.xlsx')
df.drop(columns=['Unnamed: 0'],inplace=True)
df


FileNotFoundError: [Errno 2] No such file or directory: 'df_5_toplam.xlsx'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer # Makinenin yazıyı anlayabilmesi için Tokenizer ile veri setinde kullanılan en çok 50000 kelime ile bir sözlük oluşturulur.

tokenizer = Tokenizer(num_words=50000) # sözlük oluşturulur.
tokenizer.fit_on_texts(df['Yazı']) # yazılar tokene dönüştürülür.

In [ ]:
# Verimiz için bir cümle uzunluğu hesaplanır.
 # Dizi işlemleri için numpy modülü.

# toplam token sayısı
num_tokens = [len(tokens) for tokens in df['Yazı']]
num_tokens = np.array(num_tokens)

# max token sayısı
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

2261

In [ ]:
input_dim = 50000  # Embedding layer's input dimension
embedding_dim = 100
output_dim = 5
epochs = 30
units = 512  # GRU units

In [ ]:
# Örnek veri, x_rus ve y_rus (Bu verilerin yüklenmiş ve işlenmiş olduğunu varsayıyoruz)
# Not: Gerçek veri ile değiştirin
x_rus = np.random.randint(0, input_dim, size=(10295, 2261))  # Örnek veri
y_rus = np.random.randint(output_dim, size=10295)  # Örnek veri
y_rus = tf.keras.utils.to_categorical(y_rus, num_classes=output_dim)

# Verilerin Eğitim ve Test Olarak Bölünmesi
x_train, x_test, y_train, y_test = train_test_split(x_rus, y_rus, test_size=0.25, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(7721, 2261) (7721, 5)
(2574, 2261) (2574, 5)


In [ ]:
x_train = pad_sequences(x_train, maxlen=2261, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=2261, padding='post', truncating='post')

# Giriş verisini doğrulama ve ölçeklendirme
x_train = np.clip(x_train, 0, input_dim - 1)
x_test = np.clip(x_test, 0, input_dim - 1)

In [ ]:
from tensorflow import keras # yapay sinir ağı kullanmak için keras yüklenir.
from math import exp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
#num_words kelime sayıları, embedding_size kelime vektör uzunluğu

In [ ]:
sonuçlar_df = pd.DataFrame()
import h5py
# Farklı epochs ve units kombinasyonlarını deneme
for epochs in [30]:
    for units in [512]:
        # Modeli Oluşturma
        model = Sequential([
            Embedding(input_dim=input_dim, output_dim=embedding_dim, input_length=2261),
            LSTM(units=units),
            Dense(output_dim, activation='softmax')
        ])

        # Modeli Derleme
        model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

        # Callbacks
        #callbacks = [ModelCheckpoint(filepath='mymodel.keras', monitor='val_loss', mode='min',
                                    # save_best_only=True, save_weights_only=False, verbose=1)]

        # Eğitim başlangıç zamanı
        baslangic_zamani = time.time()

        # Modeli Eğitme
        history = model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=512,
                            validation_data=(x_test, y_test), shuffle=False, verbose=0)

        # Eğitim bitiş zamanı
        bitis_zamani = time.time()
        sure = bitis_zamani - baslangic_zamani

        # Son epoch'un doğrulama kaybı
        last_val_loss = history.history['val_loss'][-1]
        print(f'Last Epoch Validation Loss = {last_val_loss}')

        # Modeli Değerlendirme
        loss = model.evaluate(x_test, y_test, batch_size=64)
        print(f"\nepochs={epochs}, units={units} => test loss: {loss}")

        # Tahminleri Al
        eğitim_predict = model.predict(x_train)
        test_predict = model.predict(x_test)

        # MSE ve RMSE Hesaplama
        mean_squared_error_test = mean_squared_error(np.argmax(y_test, axis=1), np.argmax(test_predict, axis=1))
        test_rmse = np.sqrt(mean_squared_error_test)

        mean_squared_error_eğitim = mean_squared_error(np.argmax(y_train, axis=1), np.argmax(eğitim_predict, axis=1))
        eğitim_rmse = np.sqrt(mean_squared_error_eğitim)

        # Sonuçları DataFrame'e Ekleme
        sonuçlar_df = pd.concat([sonuçlar_df, pd.DataFrame({
            'epochs': [epochs],
            'units': [units],
            'mean_squared_error_test': [mean_squared_error_test],
            'mean_squared_error_eğitim': [mean_squared_error_eğitim],
            'test_rmse': [test_rmse],
            'eğitim_rmse': [eğitim_rmse],
            'val_loss': [last_val_loss],
            'sure': [sure]
        })], ignore_index=True)

C:\anacond\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
print("Doğruluk Skoru:", accuracy)

In [ ]:
loss= model.evaluate(x_test, y_test)

In [ ]:
# Tahminler ve gerçek etiketleri bir DataFrame'e yerleştirin


# Doğruluk skorunu DataFrame'e ekleyin
accuracy_df = pd.DataFrame({'Doğruluk Skoru': [accuracy]})

sonuçlar_df=pd.concat([sonuçlar_df, accuracy_df], axis=1)

# DataFrame'leri bir Excel dosyasına yazın
with pd.ExcelWriter('/content/LSTM_results.xlsx') as writer:
    sonuçlar_df.to_excel(writer, index=False)

print("Sonuçlar Excel dosyasına kaydedildi.")

In [ ]:
from google.colab import files
files.download('/content/LSTM_results.xlsx')

In [ ]:
sonuçlar_df